In [33]:
import scanpy as sc
import numpy as np
import pandas as pd
import glob
import os

In [ ]:
seq = sc.read('/projects/Alex/Spatiotemporal_Atlas_Ecker/data/AvrRpt2_alone.h5ad')
atac = sc.read('/projects/Alex/Spatiotemporal_Atlas_Ecker/data/AvrRpt2_alone2_atac.h5ad')
chromvar = pd.read_csv('/projects/Alex/Spatiotemporal_Atlas_Ecker/data/AvrRpt2_alone2_chromvar.csv', index_col=0)

In [35]:
chromvar_ad = sc.AnnData(X=chromvar.T, var=pd.DataFrame(index=chromvar.index.values), obs=pd.DataFrame(index=chromvar.columns))

atac_names = atac.var.index.values
atac_names = ['ATAC_'+ i for i in atac_names]
atac.var.index = atac_names

chromvar_names = chromvar_ad.var.index.values
chromvar_names = ['Chromvar_'+ i for i in chromvar_names]
chromvar_ad.var.index = chromvar_names
concatenated = sc.concat([seq, atac, chromvar_ad], axis=1, merge='first')
concatenated.obs = concatenated.obs[['sample','nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'nCount_SCT', 'nFeature_SCT','percent.mt', 'time', 'doublet', 'SCT_snn_res.1',
       'nCount_peaks', 'nFeature_peaks', 'nCount_atacRNA_400bp', 'nFeature_atacRNA_400bp', 'celltype','modality']]
concatenated.obs['Cluster'] = concatenated.obs['SCT_snn_res.1'].values

del concatenated.obs['SCT_snn_res.1']
for key in ['X_harmony.rna', 'X_harmony.rna.kt57', 'X_pca', 'X_pca_kt57']:
    del concatenated.obsm[key]

In [36]:
fls = glob.glob('/projects/Alex/Spatiotemporal_Atlas_Ecker/data/integration/*')
fls = [i for i in fls if 'dc3000' not in i]

In [37]:
batches = []
for i in concatenated.obs['time']:
    if i == '04h':
        batches.append('4hr_avr')
    elif i == '06h':
        batches.append('6hr_avr')
    elif i == '09h':
        batches.append('9hr_avr')
    elif i == '24h':
        batches.append('avrrpt24')
    elif i == 'Mock':
        batches.append('mock')
    else:
        print(i)

In [38]:
concatenated.obs['batch'] = batches

In [ ]:
for fl in fls:
    ad_sc = sc.read(os.path.join(fl, 'seq_data.h5ad'))
    ad_sc.obs = ad_sc.obs[['nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'nCount_SCT', 'nFeature_SCT', 'percent.mt', 'sample', 'time', 'doublet', 'SCT_snn_res.1', 'celltype', 'dpt_pseudotime']]

    ad_sc.obs['Cluster'] = ad_sc.obs['SCT_snn_res.1'].values

    del ad_sc.obs['SCT_snn_res.1']
    del ad_sc.uns
    for key in ['X_gimvi', 'X_harmony.rna', 'X_harmony.rna.kt57', 'X_pca', 'X_pca_kt57', 'X_harmony.lsi', 'X_harmony.rna', 'X_harmony.rna.kt56', 'X_lsi', 'X_pca', 'X_pca_kt56', 'X_ref.pca', 'X_wnn.umap']:
        try:
            del ad_sc.obsm[key]
        except:
            None

    ad_sc.obsm['X_umap_joint_embedding_spatial'] = ad_sc.obsm['X_umap_gimvi']
    ad_sc.obsm['X_umap_joint_embedding_seq'] = ad_sc.obsm['X_umap']

    del ad_sc.obsm['X_umap_gimvi']
    del ad_sc.obsm['X_umap']

    ad_sc.obs.index = ['col_'+ ad_sc.obs['sample'][p] + '_' + ad_sc.obs.index[p].split('_')[-1] for p in range(len(ad_sc.obs.index))]

    ad_sc.obs['X_umap_joint_embedding_spatial_x'] = ad_sc.obsm['X_umap_joint_embedding_spatial'][:, 0]
    ad_sc.obs['X_umap_joint_embedding_spatial_y'] = ad_sc.obsm['X_umap_joint_embedding_spatial'][:, 1]
    ad_sc.obs['X_umap_joint_embedding_seq_x'] = ad_sc.obsm['X_umap_joint_embedding_seq'][:, 0]
    ad_sc.obs['X_umap_joint_embedding_seq_y'] = ad_sc.obsm['X_umap_joint_embedding_seq'][:, 1]


    conc_base = concatenated[concatenated.obs['batch'] == os.path.basename(fl)]

    ad_sc.obs = ad_sc.obs[[i for i in ad_sc.obs.columns if i not in conc_base.obs.columns]]
    ad_sc.obs['on_merge'] = [i.split('_')[2] + i.split('_')[3] + i.split('_')[4] for i in ad_sc.obs.index]
    conc_base.obs['on_merge'] = [i.split('_')[2] + i.split('_')[3] + i.split('_')[4] for i in conc_base.obs.index]
    new_conc_base = conc_base.obs.merge(ad_sc.obs, how='left', left_on='on_merge', right_on='on_merge')

    try:
        del conc_base.obsm['X_umap']
    except:
        None

    conc_base.obsm['X_umap_joint_embedding_spatial'] = new_conc_base[['X_umap_joint_embedding_spatial_x', 'X_umap_joint_embedding_spatial_y']].values
    conc_base.obsm['X_umap_joint_embedding_seq'] = new_conc_base[['X_umap_joint_embedding_seq_x', 'X_umap_joint_embedding_seq_y']].values

    conc_base.obs = new_conc_base
    #conc_base =conc_base[~conc_base.obs['dpt_pseudotime'].isna()]
    del conc_base.obs['X_umap_joint_embedding_spatial_x']
    del conc_base.obs['X_umap_joint_embedding_spatial_y']
    del conc_base.obs['X_umap_joint_embedding_seq_x']
    del conc_base.obs['X_umap_joint_embedding_seq_y']
    del conc_base.obs['on_merge']

    conc_base.write(os.path.join('/home/amonell/Desktop/arabidopsis_website_cellxgene/static/spatial_objects', 'seq_'+os.path.basename(fl)+'.h5ad'))